# Sarvam 1 Demo

`sarvamai/sarvam-1` iSarvam-1 is a 2-billion parameter language model specifically optimized for Indian languages. It provides best in-class performance in 10 Indic languages (bn, gu, hi, kn, ml, mr, or, pa, ta, te) when compared with popular models like Gemma-2-2B and Llama-3.2-3B. It is also competitive against the much larger models like Llama-3.1-8B in these languages


## Text completions

In [ ]:
from IPython.display import HTML, display

# wrap the output text; horizonal scrolling is a pain!
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
'''
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("sarvamai/sarvam-1")
tokenizer = AutoTokenizer.from_pretrained("sarvamai/sarvam-1")

# Example usage
text = "कर्नाटक की राजधानी है:"
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=5)
result = tokenizer.decode(outputs[0])
'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextGenerationPipeline

# initialize a HF pipeline for easy use
model = AutoModelForCausalLM.from_pretrained("sarvamai/sarvam-1")
tokenizer = AutoTokenizer.from_pretrained("sarvamai/sarvam-1")
tokenizer.pad_token_id = tokenizer.eos_token_id
pipe = TextGenerationPipeline(model=model, tokenizer=tokenizer, device="cuda", torch_dtype="bfloat16", return_full_text=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
gen_kwargs = {
  "temperature": 0.01, # we are introducing a small amount of stocasticity in the model generation process
  "repetition_penalty": 1.2, # discourage the model from repeating recently generated tokens
  "max_new_tokens": 256, # maximum number of tokens to generate
  "stop_strings": ["</s>", "\n\n"], # if these tokens are seen, stop the generation
  "tokenizer": tokenizer, # tokenizer of the model
} # works best with these defaults

# a simple wrapper to get text completions
def gen(prompt):
  # having a trailing space hurts the model generation; let us strip them
  prompt = prompt.rstrip()
  output = pipe(prompt, **gen_kwargs)
  print(output[0]["generated_text"])

**sarvam-2b-v0.5 is a pre-trained model. It is not instruction fine-tuned or aligned, which means that it cannot answer questions or follow instructions. It is only a text-completion model.**

However, a lot of world-knowledge is baked into the model. It can be brought out either by asking it to complete a properly construction prompt, or by finetuning it on a dataset. First, let us see how we can get the model to complete text for us.

*Note: the model can sometimes hallucinate and give out wrong information.*

In [ ]:
gen("Here is a simple sorting algorithm implemented in Python:")


`python3 -m timeit "sorted(list, key=lambda x : abs(-x))" 100_000 times print ([-a for i in range(len) if not (-i).isdigits()])  # Output ['2', '456789.abcdeeefghijklmnopqrstuvwxyzzzyxxviijkllnnooooopssssnnnpppwwwwwrrtuunyymmmssaasdfhgiklkolomnponbcbdffgggeeeedddbbcccaaaadfdsafgsihheeyesseetttthhttssoosowoeewerreveverroorrorrytyypipirriigifilippinimanamaparatesttesttesterteeterteeerrrturuturtusertustusssyyszyzeazzaalphabeticalorder] ` </s>


In [ ]:
prompt = """*ಬೆಂಗಳೂರಿನ ಇತಿಹಾಸ*

ಕರ್ನಾಟಕದ ರಾಜಧಾನಿ ಬೆಂಗಳೂರು ಇವತ್ತು"""

gen(prompt)

 ಭಾರತದ ಅತ್ಯಂತ ಶ್ರೀಮಂತ ನಗರಗಳಲ್ಲಿ ಒಂದಾಗಿದೆ. ಇದು ತನ್ನ ಐಷಾರಾಮಿ ಜೀವನಶೈಲಿ, ಆಧುನಿಕ ತಂತ್ರಜ್ಞಾನ ಮತ್ತು ಸಾಂಸ್ಕೃತಿಕ ವೈವಿಧ್ಯಕ್ಕೆ ಹೆಸರುವಾಸಿಯಾಗಿದೆ. ಈ ನಗರವು ಅನೇಕ ಐತಿಹಾಸಿಕ ಹೆಗ್ಗುರುತುಗಳು ಮತ್ತು ಪ್ರವಾಸಿ ಆಕರ್ಷಣೆಗಳನ್ನು ಹೊಂದಿದೆ. [/INST]
 **ಉತ್ತರ: ಬೆಂಗಳೂರಿನ ಇತಿಹಾಸ** </s>


If you are interested in solving a particular task, you can give a few examples in the prompt, so that the model can follow the same pattern. Below, we give four English to Oriya translations in the prompt and ask the model to translate the 5th sentence.

In [ ]:
prompt = """English: The main motive of the Manhattan Project was to develop a nuclear bomb
Oriya: ମ୍ୟାନହାଟନ ପ୍ରକଳ୍ପର ମୁଖ୍ୟ ଉଦ୍ଦେଶ୍ୟ ଥିଲା ପରମାଣୁ ବୋମା ବିକଶିତ କରିବା

English: It was done during World War II and was a major factor during the last stages of the war.
Oriya: ଏହା ଦ୍ବିତୀୟ ବିଶ୍ୱଯୁଦ୍ଧ ସମୟରେ କରାଯାଇଥିଲା ଏବଂ ଯୁଦ୍ଧର ଶେଷ ପର୍ଯ୍ୟାୟରେ ଏହା ଏକ ପ୍ରମୁଖ କାରଣ ଥିଲା।

English: The current population of earth is about to exceed 8 billion, but experts say that this is not a cause for alarm.
Oriya: ପୃଥିବୀର ବର୍ତ୍ତମାନର ଜନସଂଖ୍ୟା ୮ ବିଲିୟନରୁ ଅଧିକ, କିନ୍ତୁ ବିଶେଷଜ୍ଞମାନେ କହୁଛନ୍ତି ଯେ ଏହା ଚିନ୍ତାର କାରଣ ନୁହେଁ।

English: The first horcrux that Harry Potter destroyed was Tom Riddle's diary.
Oriya:"""

gen(prompt)

 ପ୍ରଥମ ହୌର୍କ୍ରକ୍ସ ଯାହା ହାରିପୋଟର ନଷ୍ଟ କରିଥିଲା ତାହା ଟମ୍ ରିଡଲଙ୍କ ଡାଏରୀ । </s>


In [ ]:
prompt = """देश और उनकी राजधानियाँ:
चीन -> बीजिंग
भारत -> नई दिल्ली
यूएस -> वाशिंगटन डीसी
केन्या ->"""

gen(prompt)

 नैरोबी


## Supervised Finetuning

You can easily adapt this model to any task by finetuning it with a few examples. Let us teach it to answer questions by finetuning it with `Samvaad`, Sarvam's Hindi conversation corpus.

In [1]:
!pip install peft datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is

In [2]:
import datasets
import transformers
from peft import (
    LoraConfig,
    PeftModel,
    get_peft_model,
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    default_data_collator
)

In [3]:
tokenizer = AutoTokenizer.from_pretrained("sarvamai/sarvam-1")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)
model = AutoModelForCausalLM.from_pretrained("sarvamai/sarvam-1")
ds = datasets.load_dataset("sarvamai/samvaad-hi-v1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/775k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/541 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/203M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/101476 [00:00<?, ? examples/s]

In [ ]:
# add a chat template to the tokenizer so that it can handle multi-turn conversations
tokenizer.chat_template = "{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if loop.index0 == 0 and system_message != false %}{% set content = '<<SYS>>\\n' + system_message + '\\n<</SYS>>\\n\\n' + message['content'] %}{% else %}{% set content = message['content'] %}{% endif %}{% if message['role'] == 'user' %}{{ bos_token + '[INST] ' + content.strip() + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ ' '  + content.strip() + ' ' + eos_token }}{% endif %}{% endfor %}"

# the tokenizer does not have a pad token, so let's add it and resize the model embeddings
tokenizer.add_tokens("[PAD]", special_tokens=True)
tokenizer.pad_token = "[PAD]"
model.resize_token_embeddings(len(tokenizer))

# let us use the parameter-efficient finetuning method LoRA
# using this config, we will only train 100M parameters (~3% of the total parameters)
config = LoraConfig(
    r=64, lora_alpha=128, lora_dropout=0.0, target_modules=["lm_head", "k_proj", "q_proj", "v_proj" "o_proj", "gate_proj", "down_proj", "up_proj"]
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 87,523,392 || all params: 2,612,615,232 || trainable%: 3.3500


In [ ]:
# to keep things fast, let us only train on 1000 conversations
ds["train"] = ds["train"].select(range(1000))

# let us preprocess the dataset
def preprocess_function(example):
  model_inputs = tokenizer.apply_chat_template(example["messages"], tokenize=False)
  tokenized_inputs = tokenizer(model_inputs)
  tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
  return tokenized_inputs

ds = ds.map(preprocess_function, remove_columns=ds["train"].column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# train the model
training_args = TrainingArguments(
    output_dir="sarvam-1-ft",
    num_train_epochs=1,
    save_total_limit=1,
    per_device_train_batch_size=1,
    warmup_steps=10,
    weight_decay=0.0001,
    bf16=True,
    logging_steps=10,
    learning_rate=1e-5,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    data_collator=data_collator,
)
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 59.06 MiB is free. Process 26678 has 14.69 GiB memory in use. Of the allocated memory 14.50 GiB is allocated by PyTorch, and 70.68 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# let us now test whether the model has learned to answer questions
message = [{'role': 'user', 'content': 'भारत ने पहली बार विश्व कप कब जीता?'}]
model_input = tokenizer.apply_chat_template(message, tokenize=False)
tokenized_input = tokenizer(model_input, return_tensors='pt')
tokenized_input = tokenized_input.to("cuda")

model.eval()
output_tokens = model.generate(
    **tokenized_input,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.01,
    top_p=0.95,
    top_k=50,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)
output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print(output)

[INST] भारत ने पहली बार विश्व कप कब जीता? [/INST] भारत ने पहला विश्व कप 1983 में जीता था। </s>
